In [43]:
import numpy as np
import pandas as pd
import glob, os
pd.set_option('display.max_rows', 500)

# Sport - NIBRS Join

[Football poll data](#poll)\
[Football / basketball data alignment](#alignment)\
[Spark SQL join](#join)\
[Checking full DataFrame](#check)

<a id='poll'></a>


&nbsp;

## Football poll data


In [44]:
# create DataFrame 'df' from cfbd .json files, sourced from their /games API endpoint - one .json for each year
df = pd.concat(map(lambda file: pd.read_json(file), 
                   glob.glob(os.path.join('', r'../02_sport_rawdata/cfbd_games_json/' + '*.json'))))

# create DataFrame 'df_poll' from cfbd .json files, sourced from their /rankings API endpoint - one .json for each year
df_poll = pd.concat(map(lambda file: pd.read_json(file, orient='records'), 
                        glob.glob(os.path.join('', r'../02_sport_rawdata/cfbd_poll_json/' + '*.json'))))
df_poll.reset_index(inplace=True, drop=True)


df_lst = []

def GetRankings(poll_data):
    ''' 
    This function takes a row of data and determines from which poll to extract rankings.
    There are 5 polls from which to select for a team's ranking on any given week.
    Note: This is not scaleable for large data due to looping of rows.
    '''
    
    tmp_df = pd.DataFrame(poll_data['polls'])

    search_term = ""
    if poll_data['season'] > 2013:
        if poll_data['week'] < 10: 
            search_term = 'AP Top 25'
        else:
            search_term = 'Playoff Committee'
        
    elif poll_data['season'] == 2013:
        if poll_data['week'] < 9:
            search_term = 'AP Top 25'
        else:
            search_term = 'BCS'
    else:
        if poll_data['week'] < 8:
            search_term = 'AP Top 25'
        else:
            search_term = 'BCS'
    
    if search_term == "":
        search_term = 'AP Top 25 i'
    rank_type = tmp_df[tmp_df['poll'].str.contains(search_term) == True]
    
    if rank_type.shape[0] == 0:
        search_term = 'AP Top 25'
        rank_type = tmp_df[tmp_df['poll'].str.contains('AP') == True]
    try:
        rankings = pd.json_normalize(rank_type.iloc[0]['ranks']).explode(['rank', 'school', 'conference'])
        # temp_ap = pd.json_normalize(r['polls']['poll']['ranks']).explode(['rank','school','conference'])
    except Exception:
        print(poll_data['season'], poll_data['week'], search_term)
        quit()
    
    rankings['season'] = poll_data['season']
    rankings['season_type'] = poll_data['seasonType']
    rankings['week'] = poll_data['week']
    rankings['poll'] = search_term
    
    df_lst.append(rankings)

# We don't want to return anything when we loop through the rows 
# as there will be a list of DataFrames stored we'll use in subsequent cells.
_ = df_poll.apply(lambda r: GetRankings(r),axis=1)

In [5]:
# Create the dataframe from the list of frames and validate the different polls that we're using and use some basic manipulation
cmb_ranks = pd.concat(df_lst)
cmb_ranks.poll.value_counts()
cmb_ranks['school'] = cmb_ranks['school'].str.lower()

#creating a duplicate column so that I can use that when merging for opponent ranks
cmb_ranks['opponent'] = cmb_ranks['school']
cmb_ranks

,rank,school,conference,firstPlaceVotes,points,season,season_type,week,poll,opponent
0,25,wisconsin,Big Ten,0,0,2009,regular,15,BCS,wisconsin
1,10,iowa,Big Ten,0,0,2009,regular,15,BCS,iowa
2,8,ohio state,Big Ten,0,0,2009,regular,15,BCS,ohio state
3,24,usc,Pac-10,0,0,2009,regular,15,BCS,usc
4,21,stanford,Pac-10,0,0,2009,regular,15,BCS,stanford
...,...,...,...,...,...,...,...,...,...,...
20,21,cincinnati,American Athletic,0,0,2019,regular,16,Playoff Committee,cincinnati
21,22,usc,Pac-12,0,0,2019,regular,16,Playoff Committee,usc
22,23,navy,American Athletic,0,0,2019,regular,16,Playoff Committee,navy
23,24,virginia,ACC,0,0,2019,regular,16,Playoff Committee,virginia


In [6]:
df.columns

Index(['id', 'season', 'week', 'season_type', 'start_date', 'start_time_tbd',
       'neutral_site', 'conference_game', 'attendance', 'venue_id', 'venue',
       'home_id', 'home_team', 'home_conference', 'home_points',
       'home_line_scores', 'home_post_win_prob', 'home_pregame_elo',
       'home_postgame_elo', 'away_id', 'away_team', 'away_conference',
       'away_points', 'away_line_scores', 'away_post_win_prob',
       'away_pregame_elo', 'away_postgame_elo', 'excitement_index',
       'highlights', 'notes'],
      dtype='object')

<a id='alignment'></a>


&nbsp;

## Football / basketball data alignment


The football data is not specific to a school but rather a record of games.\
\
The Basketball data is generated by school and thus we have two representations of the same game based on the school.\
\
We need to translate the football data to a similar style.

In [7]:
def DetermineResult(h_a, home_points, away_points):
    if h_a == 'H':
        if home_points > away_points:
            return 'W'
        elif home_points < away_points:
            return 'L'
        else:
            return 'T'
    else:
        if home_points > away_points:
            return 'L'
        elif home_points < away_points:
            return 'W'
        else:
            return 'T'


# reduce 'df' to football games Michigan played
um = df[(df['home_team'] =='Michigan') | (df['away_team']=='Michigan')].copy()
um['school'] = 'michigan'
um['opponent'] = um.apply(lambda row: row['away_team'] if row['home_team']=='Michigan' else row['home_team'], axis=1)
um['h_a'] = um.apply(lambda row: 'H' if row['home_team']=='Michigan' else 'A', axis=1)

# reduce 'df' to football games Michigan State played
msu = df[(df['home_team']=='Michigan State') | (df['away_team']=='Michigan State')].copy()
msu['school'] = 'michigan state'
msu['opponent'] = msu.apply(lambda row: row['away_team'] if row['home_team']=='Michigan State' else row['home_team'], axis=1)
msu['h_a'] = msu.apply(lambda row: 'H' if row['home_team']=='Michigan State' else 'A', axis=1)

um_msu = pd.concat([um, msu]) # concatenate 'um', 'msu' together for football DataFrame
um_msu['sport'] = 'football'

um_msu['result'] = um_msu.apply(lambda r: DetermineResult(r['h_a'], r['home_points'], r['away_points']), axis=1)
um_msu['team_points'] = um_msu.apply(lambda r: r['home_points'] if r['h_a'] == 'H' else r['away_points'], axis=1)
um_msu['opp_points'] = um_msu.apply(lambda r: r['home_points'] if r['h_a'] == 'A' else r['away_points'], axis=1)
um_msu['ot'] = um_msu.apply(lambda r: None if (len(r['home_line_scores']) -  4) == 0 else "OT" , axis=1)

um_msu = um_msu.merge(cmb_ranks[['rank','school','season','season_type','week']], how='left', on=['school','season','week','season_type'])
cmb_ranks['opponent_rank'] = cmb_ranks['rank']
um_msu.opponent = um_msu.opponent.str.lower()
um_msu = um_msu.merge(cmb_ranks[['opponent_rank','opponent','season','season_type','week']], how='left', on=['opponent', 'season', 'week', 'season_type'])
# um_msu['rank'] = um_msu['rank'].astype('int64')
# um_msu['opponent_rank'] = um_msu['opponent_rank'].astype('int64')


um_msu[um_msu['rank'].isna() == False]['rank'].unique()

# um_msu[['start_date','home_team','away_team','h_a','home_points','away_points','result','team_points','opp_points','ot']].sample(10)

array([25., 23., 22., 20., 21., 19., 18., 12., 11., 15., 24.,  8., 17.,
       14., 10.,  7.,  5.,  4.,  3.,  2.,  6., 16., 13.,  9.])

In [47]:
print(len(um_msu))

267


In [8]:
um_msu[['school','rank','opponent','opponent_rank','season','week','attendance']].sample(10)

,school,rank,opponent,opponent_rank,season,week,attendance
29,michigan,12.0,northwestern,NaN,2011,6,0.0
169,michigan state,13.0,boise state,24.0,2012,1,78709.0
21,michigan,NaN,purdue,NaN,2010,11,0.0
183,michigan state,NaN,youngstown state,NaN,2013,3,71626.0
112,michigan,14.0,northwestern,NaN,2018,5,0.0
126,michigan,16.0,penn state,7.0,2019,8,110669.0
226,michigan state,NaN,michigan,2.0,2016,9,75802.0
93,michigan,3.0,iowa,NaN,2016,11,70585.0
190,michigan state,16.0,nebraska,NaN,2013,12,90872.0
240,michigan state,12.0,ohio state,13.0,2017,11,107011.0


In [9]:
um_msu[um_msu['start_date'] == '2012-10-13T16:00:00.000Z'][['home_team','away_team','away_line_scores','ot']]

,home_team,away_team,away_line_scores,ot
175,Michigan State,Iowa,"[0, 3, 0, 10, 3, 3]",OT


In [10]:
um_msu['start_dt'] = pd.to_datetime(um_msu['start_date']) # convert to datetime
um_msu['end_inc_window'] = um_msu['start_dt'] +  + pd.to_timedelta(11, unit='h') # set incident time window from game start time

In [11]:
um_msu = um_msu[['school','sport','start_dt','season_type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window','attendance']]

In [12]:
um_msu.columns

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window', 'attendance'],
      dtype='object')

In [13]:
# Read in college basketball data and do some quick manipulation to ensure columns match.

cbb = pd.read_csv('../04_finaldata/df_basketball_final.csv', parse_dates=['start_dt', 'end_inc_window'])
cbb = cbb[['school','sport','start_dt','type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window']]
cbb = cbb.rename({'type': 'season_type'}, axis=1)
cbb['opponent'] = cbb['opponent'].str.lower()
cbb['season_type'] = cbb['season_type'].apply(lambda x: "regular" if x=='REG' else x)
cbb.columns
# fbb = spark.createDataFrame(um_msu[['school','sport','game_dt','season_type','opponent','result','team_points','opp_points','ot','end_inc_window']])

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window'],
      dtype='object')

In [14]:
# We join the football and basketball data and do some manipulation across the sports including binarizing results and OT, 
# cleaning up / naming the indexes (used for a unique key)
sports = pd.concat([um_msu, cbb])
sports['ot'] = sports.ot.replace([np.nan], [None])
sports['ot'] = sports.ot.replace({None: 0, "OT": 1, "2OT": 1})
sports['result'] = sports.result.replace({"W": 1,"L": 0})
sports = sports.reset_index(drop=True)
sports.index.names = ['game_id']
sports

,school,sport,start_dt,season_type,opponent,result,team_points,opp_points,ot,rank,opponent_rank,end_inc_window,attendance
game_id,,,,,,,,,,,,,
0,michigan,football,2009-09-05 19:30:00+00:00,regular,western michigan,1,31,7,0,NaN,NaN,2009-09-06 06:30:00+00:00,0.0
1,michigan,football,2009-09-12 19:30:00+00:00,regular,notre dame,1,38,34,0,NaN,18.0,2009-09-13 06:30:00+00:00,0.0
2,michigan,football,2009-09-19 16:00:00+00:00,regular,eastern michigan,1,45,17,0,25.0,NaN,2009-09-20 03:00:00+00:00,0.0
3,michigan,football,2009-09-26 16:00:00+00:00,regular,indiana,1,36,33,0,23.0,NaN,2009-09-27 03:00:00+00:00,0.0
4,michigan,football,2009-10-03 16:00:00+00:00,regular,michigan state,0,20,26,1,22.0,NaN,2009-10-04 03:00:00+00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,michigan-state,basketball,2019-03-21 14:45:00,NCAA,bradley,1,76,65,0,NaN,NaN,2019-03-22 00:45:00,NaN
989,michigan-state,basketball,2019-03-23 19:45:00,NCAA,minnesota,1,70,50,0,NaN,NaN,2019-03-24 05:45:00,NaN
990,michigan-state,basketball,2019-03-29 19:00:00,NCAA,louisiana state (12),1,80,63,0,NaN,12.0,2019-03-30 05:00:00,NaN


In [15]:
sports.columns # verify our columms

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window', 'attendance'],
      dtype='object')

In [16]:
# Reduce number of columns that will be used in our analysis (prior to join with incident data)
sports = sports[['school','sport','start_dt','season_type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window','attendance']]

In [17]:
sports.season_type.unique()

array(['regular', 'CTOURN', 'NCAA'], dtype=object)

In [48]:
print(len(sports)) # 726 basketball games + 267 football games

993


In [18]:
sports.to_csv('../04_finaldata/sports.csv', header=True)

<a id='join'></a>


&nbsp;

## Join Sports and Incidents via Spark SQL

Now that the sports and incident data are manipulated, we'll use Spark SQL to create a dataset of all the games and their associated incidents / offenses.\
\
We'll ultimately end up with a large table that identifies every offense and its attribution to a particular incident and game.


In [19]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('Merge') \
    .getOrCreate() 

sc = spark.sparkContext

In [20]:
# Convert the sports dataframe that was saved in the prior step into a spark Dataframe and ensure that the dates are casted correctly

sportsrdd = spark.read.csv('../04_finaldata/sports.csv', header=True, inferSchema=True)
sportsrdd = sportsrdd.withColumn('start_dt', col('start_dt').cast(TimestampType()))
sportsrdd = sportsrdd.withColumn('end_inc_window', col('end_inc_window').cast(TimestampType()))

In [21]:
sportsrdd.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- school: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- start_dt: timestamp (nullable = true)
 |-- season_type: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- result: integer (nullable = true)
 |-- team_points: integer (nullable = true)
 |-- opp_points: integer (nullable = true)
 |-- ot: integer (nullable = true)
 |-- rank: string (nullable = true)
 |-- opponent_rank: double (nullable = true)
 |-- end_inc_window: timestamp (nullable = true)
 |-- attendance: double (nullable = true)



In [22]:
#Let's run a quick test to see that the columns came over properly
test = sportsrdd.sample(withReplacement=False, fraction=.1)

In [23]:
test.show()

+-------+--------------+--------+-------------------+-----------+-----------------+------+-----------+----------+---+----+-------------+-------------------+----------+
|game_id|        school|   sport|           start_dt|season_type|         opponent|result|team_points|opp_points| ot|rank|opponent_rank|     end_inc_window|attendance|
+-------+--------------+--------+-------------------+-----------+-----------------+------+-----------+----------+---+----+-------------+-------------------+----------+
|      3|      michigan|football|2009-09-26 12:00:00|    regular|          indiana|     1|         36|        33|  0|23.0|         null|2009-09-26 23:00:00|       0.0|
|      7|      michigan|football|2009-10-24 15:30:00|    regular|       penn state|     0|         10|        35|  0|null|         13.0|2009-10-25 02:30:00|       0.0|
|     21|      michigan|football|2010-11-13 12:00:00|    regular|           purdue|     1|         27|        16|  0|null|         null|2010-11-13 23:00:00|    

In [24]:
# Let's get the incident data for UM and MSU and do the same thing with the dates
incs = spark.read.csv('../01_nibrs_rawdata/UM_MSU_Offs.csv', header=True, inferSchema=True)
incs = incs.withColumn('INCIDENT_DATE', col('INCIDENT_DATE').cast(TimestampType()))

In [25]:
# Create Spark SQL Table Views from both RDDs
sportsrdd.createOrReplaceTempView("sports")
incs.createOrReplaceTempView("incs")

In [26]:
# We first need to get a listing of every incident/offense associated to a games.

query = """

SELECT
    s.game_id,
    s.sport, 
    s.school, 
    s.start_dt, 
    s.end_inc_window,
    s.rank,
    s.season_type,
    s.opponent,
    s.opponent_rank,
    s.result,
    s.team_points,
    s.opp_points,
    s.ot,
    s.attendance, 
    i.INCIDENT_ID as incident_id,
    i.OFFENSE_ID as offense_id,
    i.OFFENSE_NAME as offense_name,
    i.INCIDENT_DATE as incident_date

FROM
    sports s,
    incs i

WHERE
    i.INCIDENT_DATE BETWEEN s.start_dt AND s.end_inc_window AND
    i.school = s.school
"""


result = spark.sql(query)

In [27]:
# Save off as a pandas dataframe
df = result.toPandas()

In [28]:
df.shape

(1172, 18)

In [29]:
df.head()

,game_id,sport,school,start_dt,end_inc_window,rank,season_type,opponent,opponent_rank,result,team_points,opp_points,ot,attendance,incident_id,offense_id,offense_name,incident_date
0,3,football,michigan,2009-09-26 12:00:00,2009-09-26 23:00:00,23.0,regular,indiana,NaN,1,36,33,0,0.0,50954556,55764025,Pocket-picking,2009-09-26 15:00:00
1,11,football,michigan,2009-11-21 12:00:00,2009-11-21 23:00:00,None,regular,ohio state,10.0,0,10,21,0,0.0,50956387,55769038,Destruction/Damage/Vandalism of Property,2009-11-21 22:00:00
2,2,football,michigan,2009-09-19 12:00:00,2009-09-19 23:00:00,25.0,regular,eastern michigan,NaN,1,45,17,0,0.0,50954526,55762040,All Other Larceny,2009-09-19 17:00:00
3,1,football,michigan,2009-09-12 15:30:00,2009-09-13 02:30:00,None,regular,notre dame,18.0,1,38,34,0,0.0,50955371,55778185,All Other Larceny,2009-09-12 17:00:00
4,6,football,michigan,2009-10-17 12:00:00,2009-10-17 23:00:00,None,regular,delaware state,NaN,1,63,6,0,0.0,50955435,55769019,Simple Assault,2009-10-17 15:00:00


In [30]:
# take a quick look to see home many unique games had some type of incident associated to them
len(df.game_id.unique())

451

Now, since we performed an SQL operation that returned results that matched our criteria, we need to join these results to the games dataset.  We'll create another table view and call it 'game_incs' and use a 
left join to the sports data.

In [31]:
result.createOrReplaceTempView("game_incs")

In [32]:
query2 = """

SELECT s.*, g.incident_id,g.offense_id, g.offense_name, g.incident_date

FROM
    sports s LEFT JOIN game_incs g
    ON s.game_id = g.game_id
    
"""

result2 = spark.sql(query2)


In [33]:
full = result2.toPandas().set_index('game_id')

In [34]:
len(full.index.unique()), full.shape

(993, (1714, 17))

<a id='check'></a>


&nbsp;

## Checking the full DataFrame


In [35]:
full.sample(20)

,school,sport,start_dt,season_type,opponent,result,team_points,opp_points,ot,rank,opponent_rank,end_inc_window,attendance,incident_id,offense_id,offense_name,incident_date
game_id,,,,,,,,,,,,,,,,,
444,michigan,basketball,2014-11-15 14:00:00,regular,hillsdale,1,92,68,0,24,NaN,2014-11-16 00:00:00,NaN,74955964.0,82004897.0,Theft From Building,2014-11-15 15:00:00
239,michigan state,football,2017-11-04 12:00:00,regular,penn state,1,27,24,0,24.0,7.0,2017-11-04 23:00:00,71605.0,94351430.0,116948360.0,Counterfeiting/Forgery,2017-11-04 16:00:00
764,michigan-state,basketball,2013-03-03 16:00:00,regular,michigan (4),0,57,58,0,9,4.0,2013-03-04 02:00:00,NaN,NaN,NaN,None,NaT
880,michigan-state,basketball,2016-03-11 18:30:00,CTOURN,ohio state,1,81,54,0,2,NaN,2016-03-12 04:30:00,NaN,NaN,NaN,None,NaT
829,michigan-state,basketball,2015-01-24 16:00:00,regular,nebraska,0,77,79,0,-,NaN,2015-01-25 02:00:00,NaN,NaN,NaN,None,NaT
585,michigan,basketball,2018-03-15 21:50:00,NCAA,montana,1,61,47,0,None,NaN,2018-03-16 07:50:00,NaN,NaN,NaN,None,NaT
702,michigan-state,basketball,2011-11-20 18:00:00,regular,little rock,1,69,47,0,-,NaN,2011-11-21 04:00:00,NaN,NaN,NaN,None,NaT
204,michigan state,football,2014-11-22 12:00:00,regular,rutgers,1,45,3,0,11.0,NaN,2014-11-22 23:00:00,70902.0,73419516.0,79483156.0,Drug/Narcotic Violations,2014-11-22 21:00:00
673,michigan-state,basketball,2010-12-07 21:00:00,regular,syracuse (8),0,58,72,0,7,8.0,2010-12-08 07:00:00,NaN,NaN,NaN,None,NaT


In [36]:
# Quick calculation to determine the number of sporting events that have an actual incident associated to them
grp = full.groupby('start_dt')[['incident_id','offense_name']].count()
grp[grp['incident_id'] > 0].shape[0] / 993

0.43202416918429004

In [37]:
full.to_csv('../04_finaldata/full_data.csv')

In [38]:
# Let's do a quick validation to make sure that the data associated ok. Take the game start_date with the highest number of incidents and check what offenses were created. Validate that we have the same number.
grp.sort_values('incident_id', ascending=False)

,incident_id,offense_name
start_dt,,
2014-10-25 15:30:00,14,14
2016-10-29 12:00:00,12,12
2017-10-21 15:30:00,12,12
2011-11-19 12:00:00,12,12
2009-10-03 12:00:00,11,11
...,...,...
2012-02-11 18:00:00,0,0
2015-11-27 17:30:00,0,0
2015-11-26 21:30:00,0,0


In [39]:
# Seeing 10-25-2014 has the highest number of offenses.  Let's see what incidents/offenses were generated
testsql = "SELECT i.school, i.INCIDENT_ID,i.OFFENSE_NAME FROM incs i WHERE i.INCIDENT_DATE BETWEEN '2014-10-25 15:30:00' AND '2014-10-26 2:30:00'"
r = spark.sql(testsql)
r.show()

+--------------+-----------+--------------------+
|        school|INCIDENT_ID|        OFFENSE_NAME|
+--------------+-----------+--------------------+
|      michigan|   74955899|Drug/Narcotic Vio...|
|michigan state|   73419507| Theft From Building|
|michigan state|   73418616|Destruction/Damag...|
|michigan state|   73418627|Destruction/Damag...|
|      michigan|   74955896| Theft From Building|
|michigan state|   73418619|      Simple Assault|
|michigan state|   73418637|      Simple Assault|
|michigan state|   73418625|      Simple Assault|
|michigan state|   73418636|      Simple Assault|
|michigan state|   73418620|      Simple Assault|
|michigan state|   73418617|  Aggravated Assault|
|michigan state|   73418618|      Simple Assault|
|      michigan|   74955897| Theft From Building|
|michigan state|   73418623|      Simple Assault|
+--------------+-----------+--------------------+



Looks like 14 offenses and most attributed on MSU's campus (assault).  This was a game that MSU and U-M played and MSU won, btw.

In [40]:
tstsql = "SELECT i.school, i.INCIDENT_ID,i.OFFENSE_NAME FROM incs i WHERE i.INCIDENT_DATE BETWEEN '2011-01-30 18:00:00' AND '2011-01-31 04:00:00'"
tresult = spark.sql(tstsql)
tresult.show()

+------+-----------+------------+
|school|INCIDENT_ID|OFFENSE_NAME|
+------+-----------+------------+
+------+-----------+------------+



In [41]:
df_full = pd.read_csv('../04_finaldata/full_data.csv', parse_dates=['start_dt', 'end_inc_window'])